# RouteTemplateWhole 
This notebook is the template route notebook where a route e.g 16 in one direction is selected from the cleaned leaving and trips data combined. Features needed for training and testing are added in this notebook, historical weather data is added and a csv needed for the prediction modelling is cerated.

In [1]:
#Import the required packages
import scipy
#Import package pandas for data analysis
import pandas as pd

#Import package numpy for numeric computing
import numpy as np

#Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#Import package seaborn for visualisation
import seaborn as sns

#For showing plots directly in the notebook run the command below
%matplotlib inline

from matplotlib.backends.backend_pdf import PdfPages

#Import all prediction packages
import sqlalchemy
from sqlalchemy import create_engine
from pandas import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.metrics import r2_score
from sklearn import tree
import matplotlib.pyplot as plt
import pickle
import requests
import json
import numpy as np
import datetime
import time
from time import gmtime
from time import strftime


import warnings
warnings.filterwarnings('ignore')

## 1. Reading in data
This section involves reading in MasterCleanedLeaving&TripsDataCombined.feather which came from notebooks Cleaned_leavetime_data and Cleaned_trips_data and choosing a specific route i.e 16 in one direction.

In [ ]:
#read in feather file
df = pd.read_feather("MasterCleanedLeaving&TripsDataCombined.feather")

In [3]:
#get df of route 16 only
df_16 = df.loc[df['LINEID'] == '16']

In [4]:
#get route 16 in direction 1 only
df_16_1 = df_16.loc[df_16['DIRECTION'] == 1]

In [5]:
df_16[df_16['DIRECTION']==2]

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,ArrDel,DepDel,Waiting_Time
142458,93868,2018-01-01,5968866,15,1328,67665,67665,67602,67622,1000895,16,16_24,2,63,43,20
142459,93900,2018-01-01,5968866,16,1329,67739,67739,67667,67683,1000895,16,16_24,2,72,56,16
142460,93933,2018-01-01,5968866,17,1330,67771,67771,67707,67707,1000895,16,16_24,2,64,64,0
142461,93949,2018-01-01,5968866,51,17,70080,70080,70710,70778,1000895,16,16_24,2,-630,-698,68
142462,94019,2018-01-01,5968866,53,19,70177,70177,70878,70888,1000895,16,16_24,2,-701,-711,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103088559,116689173,2018-12-30,8576046,8,2980,78503,78503,78464,78464,1001169,16,16_24,2,39,39,0
103088560,116689352,2018-12-30,8576046,11,2992,78573,78573,78522,78522,1001169,16,16_24,2,51,51,0
103088561,116689407,2018-12-30,8576046,12,1325,78594,78594,78540,78540,1001169,16,16_24,2,54,54,0
103088562,116689463,2018-12-30,8576046,34,1347,79245,79245,79217,79217,1001169,16,16_24,2,28,28,0


In [6]:
df_16_1.tail()

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,ArrDel,DepDel,Waiting_Time
103088815,116689043,2018-12-30,8588850,61,1304,49859,49859,50456,50456,1000955,16,16_20,1,-597,-597,0
103088816,116689095,2018-12-30,8588850,62,1305,49911,49911,50487,50494,1000955,16,16_20,1,-576,-583,7
103088817,116692775,2018-12-30,8588850,8,1634,46219,46219,46520,46520,1000955,16,16_20,1,-301,-301,0
103088818,116692828,2018-12-30,8588850,9,1635,46232,46232,46530,46530,1000955,16,16_20,1,-298,-298,0
103088819,116692868,2018-12-30,8588850,32,49,47614,47614,48116,48143,1000955,16,16_20,1,-502,-529,27


In [7]:
#need to sort dataframe by TRIPID, DAYOFSERVICE and PROGRNUMBER
df_16_1 = df_16_1.sort_values(by=['TRIPID', 'DAYOFSERVICE','PROGRNUMBER'], ascending = [False,True,True])

In [8]:
df_16_1

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,ArrDel,DepDel,Waiting_Time
102142973,114877794,2018-12-27,8591828,1,7347,56880,56880,56891,56891,1000115,16,16_20,1,-11,-11,0
102142927,114812203,2018-12-27,8591828,2,3669,56950,56950,56972,57002,1000115,16,16_20,1,-22,-52,30
102142928,114812204,2018-12-27,8591828,3,7349,57003,57003,57085,57085,1000115,16,16_20,1,-82,-82,0
102142981,114886799,2018-12-27,8591828,4,1631,57068,57068,57166,57166,1000115,16,16_20,1,-98,-98,0
102142985,114891220,2018-12-27,8591828,5,1632,57086,57086,57173,57173,1000115,16,16_20,1,-87,-87,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118942,130366,2018-01-01,5955277,72,2970,33769,33769,33016,33016,1001127,16,16_20,1,753,753,0
118943,130639,2018-01-01,5955277,73,2971,33808,33808,33034,33034,1001127,16,16_20,1,774,774,0
118944,130671,2018-01-01,5955277,74,5051,33867,33867,33108,33117,1001127,16,16_20,1,759,750,9
118916,125695,2018-01-01,5955277,75,5052,33892,33892,33149,33149,1001127,16,16_20,1,743,743,0


## 2. Adding Features

This section involves adding the features needed for prediction modelling. Creating the priorstops arrival time feature is needed for the reation of the JourneyTime feature.

In [9]:
dates = []
for row in df_16_1['DAYOFSERVICE']:
    dates.append(row)
    
dates

[Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-27 00:00:00'),
 Timestamp('2018-12-

In [10]:
df_16_1.dtypes

index                       int64
DAYOFSERVICE       datetime64[ns]
TRIPID                      int64
PROGRNUMBER              category
STOPPOINTID              category
PLANNEDTIME_ARR             int64
PLANNEDTIME_DEP             int64
ACTUALTIME_ARR              int64
ACTUALTIME_DEP              int64
VEHICLEID                category
LINEID                     object
ROUTEID                    object
DIRECTION                   int64
ArrDel                      int64
DepDel                      int64
Waiting_Time                int64
dtype: object

In [11]:
df_16_1.PROGRNUMBER

102142973     1
102142927     2
102142928     3
102142981     4
102142985     5
             ..
118942       72
118943       73
118944       74
118916       75
118917       76
Name: PROGRNUMBER, Length: 1557835, dtype: category
Categories (109, int64): [1, 2, 3, 4, ..., 106, 107, 108, 109]

In [12]:
priorstops = df_16_1['PROGRNUMBER']

In [13]:
priorstops

102142973     1
102142927     2
102142928     3
102142981     4
102142985     5
             ..
118942       72
118943       73
118944       74
118916       75
118917       76
Name: PROGRNUMBER, Length: 1557835, dtype: category
Categories (109, int64): [1, 2, 3, 4, ..., 106, 107, 108, 109]

In [14]:
#creating the array of priorstops
priorstops = np.array(priorstops[:-1]).astype(int)

In [15]:
priorstops

array([ 1,  2,  3, ..., 73, 74, 75])

### Creating priorstops actual arrival time feature

In [16]:
# Creating actual arrival time of prior stopid
priorstops_actualTime_Arr = df_16_1['ACTUALTIME_ARR']
priorstops_actualTime_Arr = np.array(priorstops_actualTime_Arr[:-1])

In [17]:
priorstops_actualTime_Arr

array([56891, 56972, 57085, ..., 33034, 33108, 33149])

In [18]:
priorstops_actualTime_Arr = np.insert(priorstops_actualTime_Arr, 0, priorstops_actualTime_Arr[0])

In [19]:
priorstops_actualTime_Arr

array([56891, 56891, 56972, ..., 33034, 33108, 33149])

In [20]:
df_16_1['priorstops_actualTime_Arr'] = priorstops_actualTime_Arr

In [21]:
df_16_1

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,LINEID,ROUTEID,DIRECTION,ArrDel,DepDel,Waiting_Time,priorstops_actualTime_Arr
102142973,114877794,2018-12-27,8591828,1,7347,56880,56880,56891,56891,1000115,16,16_20,1,-11,-11,0,56891
102142927,114812203,2018-12-27,8591828,2,3669,56950,56950,56972,57002,1000115,16,16_20,1,-22,-52,30,56891
102142928,114812204,2018-12-27,8591828,3,7349,57003,57003,57085,57085,1000115,16,16_20,1,-82,-82,0,56972
102142981,114886799,2018-12-27,8591828,4,1631,57068,57068,57166,57166,1000115,16,16_20,1,-98,-98,0,57085
102142985,114891220,2018-12-27,8591828,5,1632,57086,57086,57173,57173,1000115,16,16_20,1,-87,-87,0,57166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118942,130366,2018-01-01,5955277,72,2970,33769,33769,33016,33016,1001127,16,16_20,1,753,753,0,33002
118943,130639,2018-01-01,5955277,73,2971,33808,33808,33034,33034,1001127,16,16_20,1,774,774,0,33016
118944,130671,2018-01-01,5955277,74,5051,33867,33867,33108,33117,1001127,16,16_20,1,759,750,9,33034
118916,125695,2018-01-01,5955277,75,5052,33892,33892,33149,33149,1001127,16,16_20,1,743,743,0,33108


### Creating Journey Time Target Feature

This feature is derived from subtracting the 

In [22]:
df_16_1['JourneyTime'] = df_16_1['ACTUALTIME_DEP'] - df_16_1['priorstops_actualTime_Arr']

In [23]:
df_16_1['JourneyTime']

102142973      0
102142927    111
102142928    113
102142981     81
102142985      7
            ... 
118942        14
118943        18
118944        83
118916        41
118917        35
Name: JourneyTime, Length: 1557835, dtype: int64

### Formatting Timestamps Section
Changing timestamps to datetime format for all timestamps and adding the datetime format to the dataframe

In [24]:
df_16_1["ACTUALTIME_ARR"]

102142973    56891
102142927    56972
102142928    57085
102142981    57166
102142985    57173
             ...  
118942       33016
118943       33034
118944       33108
118916       33149
118917       33184
Name: ACTUALTIME_ARR, Length: 1557835, dtype: int64

In [25]:
pd.Timedelta(56891)

Timedelta('0 days 00:00:00.000056891')

In [26]:
gmtime(56891)

time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=15, tm_min=48, tm_sec=11, tm_wday=3, tm_yday=1, tm_isdst=0)

In [27]:
actualTime_Arr = []

for row in df_16_1['ACTUALTIME_ARR']:
    time = strftime("%H:%M:%S", gmtime(row))
    actualTime_Arr.append(time)
            
actualTime_Arr

['15:48:11',
 '15:49:32',
 '15:51:25',
 '15:52:46',
 '15:52:53',
 '15:53:05',
 '15:53:23',
 '15:55:10',
 '15:55:24',
 '15:55:59',
 '15:58:05',
 '15:58:33',
 '15:59:50',
 '16:01:20',
 '16:03:22',
 '16:04:42',
 '16:06:02',
 '16:06:19',
 '16:07:04',
 '16:07:16',
 '16:09:10',
 '16:09:31',
 '16:09:56',
 '16:10:18',
 '16:10:59',
 '16:11:26',
 '16:12:00',
 '16:12:36',
 '16:13:39',
 '16:14:47',
 '16:15:12',
 '16:16:35',
 '16:18:49',
 '16:19:15',
 '16:21:22',
 '16:26:55',
 '16:32:03',
 '16:36:08',
 '16:39:20',
 '16:41:53',
 '16:43:14',
 '16:44:48',
 '16:47:23',
 '16:48:38',
 '16:49:24',
 '16:51:57',
 '16:53:05',
 '16:53:54',
 '16:54:44',
 '16:55:39',
 '16:58:18',
 '16:59:06',
 '17:00:08',
 '17:00:45',
 '17:01:32',
 '17:02:13',
 '17:02:46',
 '17:03:11',
 '17:03:51',
 '17:04:24',
 '17:05:48',
 '17:08:07',
 '17:08:59',
 '17:09:55',
 '17:10:18',
 '17:11:23',
 '17:12:11',
 '17:12:39',
 '17:13:35',
 '17:13:56',
 '17:14:27',
 '17:14:51',
 '17:15:47',
 '17:17:07',
 '17:17:36',
 '17:21:48',
 '15:46:30',

In [28]:
df_16_1['ACTUALTIME_ARR'] = actualTime_Arr

In [29]:
actualTime_Dep = []

for row in df_16_1['ACTUALTIME_DEP']:
    time = strftime("%H:%M:%S", gmtime(row))
    actualTime_Dep.append(time)
            
actualTime_Dep

['15:48:11',
 '15:50:02',
 '15:51:25',
 '15:52:46',
 '15:52:53',
 '15:53:05',
 '15:53:56',
 '15:55:10',
 '15:55:24',
 '15:55:59',
 '15:58:05',
 '15:58:42',
 '16:00:06',
 '16:01:59',
 '16:03:22',
 '16:04:42',
 '16:06:02',
 '16:06:34',
 '16:07:04',
 '16:07:16',
 '16:09:10',
 '16:09:31',
 '16:09:56',
 '16:10:18',
 '16:10:59',
 '16:11:38',
 '16:12:00',
 '16:12:36',
 '16:14:09',
 '16:14:47',
 '16:15:26',
 '16:17:30',
 '16:18:49',
 '16:19:15',
 '16:25:28',
 '16:28:33',
 '16:33:32',
 '16:36:53',
 '16:40:32',
 '16:42:42',
 '16:43:37',
 '16:45:29',
 '16:47:33',
 '16:48:55',
 '16:49:34',
 '16:52:11',
 '16:53:19',
 '16:54:08',
 '16:54:57',
 '16:55:50',
 '16:58:33',
 '16:59:27',
 '17:00:19',
 '17:00:58',
 '17:01:46',
 '17:02:13',
 '17:02:46',
 '17:03:23',
 '17:04:06',
 '17:04:38',
 '17:05:48',
 '17:08:24',
 '17:09:18',
 '17:09:55',
 '17:10:58',
 '17:11:37',
 '17:12:11',
 '17:12:52',
 '17:13:35',
 '17:14:04',
 '17:14:27',
 '17:14:59',
 '17:15:47',
 '17:17:07',
 '17:17:47',
 '17:21:48',
 '15:46:30',

In [30]:
df_16_1['ACTUALTIME_DEP'] = actualTime_Dep

In [31]:
PlannedTime_Arr = []

for row in df_16_1['PLANNEDTIME_ARR']:
    time = strftime("%H:%M:%S", gmtime(row))
    PlannedTime_Arr.append(time)
            
PlannedTime_Arr

['15:48:00',
 '15:49:10',
 '15:50:03',
 '15:51:08',
 '15:51:26',
 '15:51:49',
 '15:52:11',
 '15:53:04',
 '15:53:23',
 '15:54:01',
 '15:55:19',
 '15:55:47',
 '15:57:30',
 '15:58:59',
 '16:00:02',
 '16:02:09',
 '16:03:09',
 '16:03:50',
 '16:04:58',
 '16:05:36',
 '16:07:31',
 '16:08:22',
 '16:09:22',
 '16:10:23',
 '16:11:28',
 '16:12:31',
 '16:13:16',
 '16:14:08',
 '16:15:16',
 '16:16:38',
 '16:17:45',
 '16:19:21',
 '16:20:50',
 '16:21:38',
 '16:24:36',
 '16:28:08',
 '16:32:08',
 '16:35:18',
 '16:38:13',
 '16:40:21',
 '16:41:13',
 '16:43:00',
 '16:46:34',
 '16:47:32',
 '16:48:28',
 '16:49:44',
 '16:51:30',
 '16:52:51',
 '16:54:08',
 '16:55:09',
 '16:56:24',
 '16:57:16',
 '16:58:21',
 '16:58:55',
 '17:00:13',
 '17:01:25',
 '17:02:41',
 '17:03:14',
 '17:04:00',
 '17:04:31',
 '17:05:30',
 '17:06:22',
 '17:07:07',
 '17:07:41',
 '17:08:13',
 '17:08:48',
 '17:09:44',
 '17:10:28',
 '17:11:03',
 '17:11:37',
 '17:12:09',
 '17:12:44',
 '17:13:17',
 '17:14:07',
 '17:14:49',
 '17:15:31',
 '15:48:00',

In [32]:
df_16_1['PLANNEDTIME_ARR'] = PlannedTime_Arr

In [33]:
PlannedTime_Dep = []

for row in df_16_1['PLANNEDTIME_DEP']:
    time = strftime("%H:%M:%S", gmtime(row))
    PlannedTime_Dep.append(time)
            
PlannedTime_Dep

['15:48:00',
 '15:49:10',
 '15:50:03',
 '15:51:08',
 '15:51:26',
 '15:51:49',
 '15:52:11',
 '15:53:04',
 '15:53:23',
 '15:54:01',
 '15:55:19',
 '15:55:47',
 '15:57:30',
 '15:58:59',
 '16:00:02',
 '16:02:09',
 '16:03:09',
 '16:03:50',
 '16:04:58',
 '16:05:36',
 '16:07:31',
 '16:08:22',
 '16:09:22',
 '16:10:23',
 '16:11:28',
 '16:12:31',
 '16:13:16',
 '16:14:08',
 '16:15:16',
 '16:16:38',
 '16:17:45',
 '16:19:21',
 '16:20:50',
 '16:21:38',
 '16:24:36',
 '16:28:08',
 '16:32:08',
 '16:35:18',
 '16:38:13',
 '16:40:21',
 '16:41:13',
 '16:43:00',
 '16:46:34',
 '16:47:32',
 '16:48:28',
 '16:49:44',
 '16:51:30',
 '16:52:51',
 '16:54:08',
 '16:55:09',
 '16:56:24',
 '16:57:16',
 '16:58:21',
 '16:58:55',
 '17:00:13',
 '17:01:25',
 '17:02:41',
 '17:03:14',
 '17:04:00',
 '17:04:31',
 '17:05:30',
 '17:06:22',
 '17:07:07',
 '17:07:41',
 '17:08:13',
 '17:08:48',
 '17:09:44',
 '17:10:28',
 '17:11:03',
 '17:11:37',
 '17:12:09',
 '17:12:44',
 '17:13:17',
 '17:14:07',
 '17:14:49',
 '17:15:31',
 '15:48:00',

In [34]:
df_16_1['PLANNEDTIME_DEP'] = PlannedTime_Dep

In [35]:
priorStopsActualTime_Arr = []

for row in df_16_1['priorstops_actualTime_Arr']:
    time = strftime("%H:%M:%S", gmtime(row))
    priorStopsActualTime_Arr.append(time)
            
priorStopsActualTime_Arr

['15:48:11',
 '15:48:11',
 '15:49:32',
 '15:51:25',
 '15:52:46',
 '15:52:53',
 '15:53:05',
 '15:53:23',
 '15:55:10',
 '15:55:24',
 '15:55:59',
 '15:58:05',
 '15:58:33',
 '15:59:50',
 '16:01:20',
 '16:03:22',
 '16:04:42',
 '16:06:02',
 '16:06:19',
 '16:07:04',
 '16:07:16',
 '16:09:10',
 '16:09:31',
 '16:09:56',
 '16:10:18',
 '16:10:59',
 '16:11:26',
 '16:12:00',
 '16:12:36',
 '16:13:39',
 '16:14:47',
 '16:15:12',
 '16:16:35',
 '16:18:49',
 '16:19:15',
 '16:21:22',
 '16:26:55',
 '16:32:03',
 '16:36:08',
 '16:39:20',
 '16:41:53',
 '16:43:14',
 '16:44:48',
 '16:47:23',
 '16:48:38',
 '16:49:24',
 '16:51:57',
 '16:53:05',
 '16:53:54',
 '16:54:44',
 '16:55:39',
 '16:58:18',
 '16:59:06',
 '17:00:08',
 '17:00:45',
 '17:01:32',
 '17:02:13',
 '17:02:46',
 '17:03:11',
 '17:03:51',
 '17:04:24',
 '17:05:48',
 '17:08:07',
 '17:08:59',
 '17:09:55',
 '17:10:18',
 '17:11:23',
 '17:12:11',
 '17:12:39',
 '17:13:35',
 '17:13:56',
 '17:14:27',
 '17:14:51',
 '17:15:47',
 '17:17:07',
 '17:17:36',
 '17:21:48',

In [36]:
df_16_1['priorstops_actualTime_Arr'] = priorStopsActualTime_Arr

### Adding Month, Day, Hour Features
This section adds any necessary features needed for the prediction and for the merging of the weather data.

In [37]:
df_16_1['Month'] = df_16_1['DAYOFSERVICE'].dt.month


In [38]:
df_16_1['Day'] = df_16_1['DAYOFSERVICE'].dt.dayofweek

In [39]:
df_16_1['Hour'] = df_16_1['ACTUALTIME_ARR'].apply(pd.to_datetime).dt.hour

KeyboardInterrupt: 

### Adding 1 if school is still in session i.e any other months but the summer months and adding 0 if the school is closed i.e in the summer months of June, July, August

In [46]:
list_School = []

for row in df_16_1['Month']:
    if row == 6 or row == 7 or row== 8:
        list_School.append('0')
    else:
        list_School.append('1')  

In [47]:
df_16_1['School']=list_School 

### Adding rushHour feature where if a journey is between the hours of 7pm and 8.30pm and 4pm and 6pm, the row gets a 0 and if not, gets a 1.

In [48]:
list_rushHour = []

for row in df_16_1['ACTUALTIME_DEP']:
    if row >= '07:00:00' and row <= '08:30:00' or row >= '16:00:00' and row <= '18:00:00':
        list_rushHour.append('1')
    else:
        list_rushHour.append('0')

In [49]:
df_16_1['RushHour']=list_rushHour

### Adding weekend feature, if a journey is on a weekend day a row gets a 1 and if not, a 0.

In [50]:
list_weekend = []

for row in df_16_1['Day']:
    if row == '5' or row == '6':
        list_weekend.append('1')
    else:
        list_weekend.append('0')

In [51]:
df_16_1['Weekend'] = list_weekend

### Adding holidays feature, if a bus journey is on a holiday date, the row gets a 1 and if not, a 0.

In [52]:
holidays = ['2018/01/01', '2018/03/17', '2018/04/02', '2018/05/07','2018/06/04', '2018/08/06', '2018/10/29', '2018/12/25', '2018/12/26']

newHolidays = []
for holiday in holidays:
    newHolidays.append(datetime.datetime.strptime(holiday,"%Y/%m/%d"))


In [53]:
newhols1 = []

for holiday in newHolidays:
      newhols1.append(holiday.date())

In [54]:
list_dfHols = []
for row in df_16_1['DAYOFSERVICE']:
#         print(row.date())
    if row.date() == datetime.date(2018, 1, 1) or row.date() == datetime.date(2018, 3, 17) or row.date() == datetime.date(2018, 4, 2) or row.date() == datetime.date(2018, 5, 7) or row.date() == datetime.date(2018, 6, 4) or row.date() == datetime.date(2018, 8, 6) or row.date() == datetime.date(2018, 10, 29) or row.date() ==  datetime.date(2018, 12, 25) or row.date() == datetime.date(2018, 12, 26):
#             print('true')
        list_dfHols.append('1')           
    else:
        list_dfHols.append('0')
#             print('false')

In [55]:
df_16_1['Holiday'] = list_dfHols

### Remove any journey times that are negative from the dataframe

In [40]:
df_16_1 = df_16_1[(df_16_1['JourneyTime'] > 0)]

In [41]:
df_16_1.describe()

,index,TRIPID,DIRECTION,ArrDel,DepDel,Waiting_Time,JourneyTime,Month,Day
count,1.538748e+06,1.538748e+06,1538748.0,1.538748e+06,1.538748e+06,1.538748e+06,1.538748e+06,1.538748e+06,1.538748e+06
mean,5.884265e+07,7.161795e+06,1.0,-2.548595e+02,-2.708486e+02,1.598913e+01,1.296740e+02,6.460380e+00,2.763083e+00
std,3.443588e+07,7.480456e+05,0.0,4.532400e+02,4.546691e+02,3.342138e+01,1.149142e+03,3.483047e+00,1.926098e+00
min,7.944200e+04,5.955277e+06,1.0,-7.796000e+03,-7.796000e+03,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00
25%,2.897173e+07,6.591171e+06,1.0,-4.640000e+02,-4.830000e+02,0.000000e+00,4.300000e+01,3.000000e+00,1.000000e+00
50%,5.820280e+07,7.106446e+06,1.0,-1.760000e+02,-1.920000e+02,1.100000e+01,6.900000e+01,6.000000e+00,3.000000e+00
75%,8.910902e+07,8.023721e+06,1.0,-2.000000e+00,-1.600000e+01,1.900000e+01,1.130000e+02,1.000000e+01,4.000000e+00
max,1.168424e+08,8.591828e+06,1.0,2.815000e+03,2.815000e+03,3.805000e+03,5.582000e+04,1.200000e+01,6.000000e+00


### Add route 16 direction 1 dataframe with all features except weather to csv

In [57]:
df_16_1.to_csv('df_16_1.csv')

## 3. Combining Weather with the route 16 direction 1 dataframe

This section involves reading in Met Eireann Historical 2018 weather data from the mySQL database and adding predictive features to the route 16 direction 1 dataframe

In [123]:
weather = pd.read_csv('historic_weather.csv')

In [124]:
weather.shape

(8760, 16)

In [125]:
weather

,weather_date,rain,temp,wetb,dewpt,vappr,rhum,msl,wdsp,wddir,ww,w,sun,vis,clht,clamt
0,2018-01-01 00:00:00,0.1,4.0,3.2,2.0,7.0,87,991.3,20,220,60,61,0.0,20000,38,7
1,2018-01-01 01:00:00,0.0,4.3,3.5,2.4,7.2,87,991.4,22,220,21,62,0.0,25000,26,7
2,2018-01-01 02:00:00,0.0,4.3,3.5,2.2,7.2,87,991.3,20,220,21,62,0.0,25000,33,6
3,2018-01-01 03:00:00,0.0,4.7,3.9,2.6,7.4,87,991.1,20,220,1,61,0.0,25000,999,3
4,2018-01-01 04:00:00,0.0,4.8,3.9,2.6,7.4,86,990.8,20,220,2,11,0.0,20000,999,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,2018-12-31 19:00:00,0.0,9.5,7.8,5.8,9.2,78,1034.8,12,250,2,11,0.0,30000,24,7
8756,2018-12-31 20:00:00,0.0,9.6,7.9,5.8,9.3,78,1035.0,13,250,2,11,0.0,25000,25,7
8757,2018-12-31 21:00:00,0.0,9.5,7.7,5.6,9.1,76,1035.1,12,250,2,11,0.0,25000,25,7
8758,2018-12-31 22:00:00,0.0,9.3,7.7,5.9,9.2,79,1035.2,11,250,2,11,0.0,25000,23,7


In [126]:
weather.isna().sum()

weather_date    0
rain            0
temp            0
wetb            0
dewpt           0
vappr           0
rhum            0
msl             0
wdsp            0
wddir           0
ww              0
w               0
sun             0
vis             0
clht            0
clamt           0
dtype: int64

In [127]:
#Adding Hour and DayOfService Feature to the weather dataframe to be able to merge with the route 16 dataframe.
weather['Hour'] = weather.weather_date.apply(pd.to_datetime).dt.hour
weather['DAYOFSERVICE'] = weather.weather_date.apply(pd.to_datetime).dt.date


In [128]:
weather.head()

,weather_date,rain,temp,wetb,dewpt,vappr,rhum,msl,wdsp,wddir,ww,w,sun,vis,clht,clamt,Hour,DAYOFSERVICE
0,2018-01-01 00:00:00,0.1,4.0,3.2,2.0,7.0,87,991.3,20,220,60,61,0.0,20000,38,7,0,2018-01-01
1,2018-01-01 01:00:00,0.0,4.3,3.5,2.4,7.2,87,991.4,22,220,21,62,0.0,25000,26,7,1,2018-01-01
2,2018-01-01 02:00:00,0.0,4.3,3.5,2.2,7.2,87,991.3,20,220,21,62,0.0,25000,33,6,2,2018-01-01
3,2018-01-01 03:00:00,0.0,4.7,3.9,2.6,7.4,87,991.1,20,220,1,61,0.0,25000,999,3,3,2018-01-01
4,2018-01-01 04:00:00,0.0,4.8,3.9,2.6,7.4,86,990.8,20,220,2,11,0.0,20000,999,2,4,2018-01-01


In [129]:
weather.dtypes

weather_date     object
rain            float64
temp            float64
wetb            float64
dewpt           float64
vappr           float64
rhum              int64
msl             float64
wdsp              int64
wddir             int64
ww                int64
w                 int64
sun             float64
vis               int64
clht              int64
clamt             int64
Hour              int64
DAYOFSERVICE     object
dtype: object

In [130]:
df_16_1.dtypes

index                                 int64
DAYOFSERVICE                 datetime64[ns]
TRIPID                                int64
PROGRNUMBER                           int64
STOPPOINTID                        category
PLANNEDTIME_ARR                      object
PLANNEDTIME_DEP                      object
ACTUALTIME_ARR                       object
ACTUALTIME_DEP                       object
VEHICLEID                          category
LINEID                               object
ROUTEID                              object
DIRECTION                             int64
ArrDel                                int64
DepDel                                int64
Waiting_Time                          int64
priorstops_actualTime_Arr            object
JourneyTime                           int64
Month                                 int64
Day                                   int64
Hour                                  int64
School                               object
RushHour                        

In [131]:
weather['rain'] = weather['rain'].astype('float')

In [132]:
df_16_1['PROGRNUMBER']=df_16_1['PROGRNUMBER'].astype('int')

In [133]:
weather['DAYOFSERVICE'] = weather['DAYOFSERVICE'].astype('datetime64[ns]')

In [134]:
for row in weather['weather_date']:
    print(row)

2018-01-01 00:00:00
2018-01-01 01:00:00
2018-01-01 02:00:00
2018-01-01 03:00:00
2018-01-01 04:00:00
2018-01-01 05:00:00
2018-01-01 06:00:00
2018-01-01 07:00:00
2018-01-01 08:00:00
2018-01-01 09:00:00
2018-01-01 10:00:00
2018-01-01 11:00:00
2018-01-01 12:00:00
2018-01-01 13:00:00
2018-01-01 14:00:00
2018-01-01 15:00:00
2018-01-01 16:00:00
2018-01-01 17:00:00
2018-01-01 18:00:00
2018-01-01 19:00:00
2018-01-01 20:00:00
2018-01-01 21:00:00
2018-01-01 22:00:00
2018-01-01 23:00:00
2018-01-02 00:00:00
2018-01-02 01:00:00
2018-01-02 02:00:00
2018-01-02 03:00:00
2018-01-02 04:00:00
2018-01-02 05:00:00
2018-01-02 06:00:00
2018-01-02 07:00:00
2018-01-02 08:00:00
2018-01-02 09:00:00
2018-01-02 10:00:00
2018-01-02 11:00:00
2018-01-02 12:00:00
2018-01-02 13:00:00
2018-01-02 14:00:00
2018-01-02 15:00:00
2018-01-02 16:00:00
2018-01-02 17:00:00
2018-01-02 18:00:00
2018-01-02 19:00:00
2018-01-02 20:00:00
2018-01-02 21:00:00
2018-01-02 22:00:00
2018-01-02 23:00:00
2018-01-03 00:00:00
2018-01-03 01:00:00


2018-03-08 04:00:00
2018-03-08 05:00:00
2018-03-08 06:00:00
2018-03-08 07:00:00
2018-03-08 08:00:00
2018-03-08 09:00:00
2018-03-08 10:00:00
2018-03-08 11:00:00
2018-03-08 12:00:00
2018-03-08 13:00:00
2018-03-08 14:00:00
2018-03-08 15:00:00
2018-03-08 16:00:00
2018-03-08 17:00:00
2018-03-08 18:00:00
2018-03-08 19:00:00
2018-03-08 20:00:00
2018-03-08 21:00:00
2018-03-08 22:00:00
2018-03-08 23:00:00
2018-03-09 00:00:00
2018-03-09 01:00:00
2018-03-09 02:00:00
2018-03-09 03:00:00
2018-03-09 04:00:00
2018-03-09 05:00:00
2018-03-09 06:00:00
2018-03-09 07:00:00
2018-03-09 08:00:00
2018-03-09 09:00:00
2018-03-09 10:00:00
2018-03-09 11:00:00
2018-03-09 12:00:00
2018-03-09 13:00:00
2018-03-09 14:00:00
2018-03-09 15:00:00
2018-03-09 16:00:00
2018-03-09 17:00:00
2018-03-09 18:00:00
2018-03-09 19:00:00
2018-03-09 20:00:00
2018-03-09 21:00:00
2018-03-09 22:00:00
2018-03-09 23:00:00
2018-03-10 00:00:00
2018-03-10 01:00:00
2018-03-10 02:00:00
2018-03-10 03:00:00
2018-03-10 04:00:00
2018-03-10 05:00:00


2018-05-18 02:00:00
2018-05-18 03:00:00
2018-05-18 04:00:00
2018-05-18 05:00:00
2018-05-18 06:00:00
2018-05-18 07:00:00
2018-05-18 08:00:00
2018-05-18 09:00:00
2018-05-18 10:00:00
2018-05-18 11:00:00
2018-05-18 12:00:00
2018-05-18 13:00:00
2018-05-18 14:00:00
2018-05-18 15:00:00
2018-05-18 16:00:00
2018-05-18 17:00:00
2018-05-18 18:00:00
2018-05-18 19:00:00
2018-05-18 20:00:00
2018-05-18 21:00:00
2018-05-18 22:00:00
2018-05-18 23:00:00
2018-05-19 00:00:00
2018-05-19 01:00:00
2018-05-19 02:00:00
2018-05-19 03:00:00
2018-05-19 04:00:00
2018-05-19 05:00:00
2018-05-19 06:00:00
2018-05-19 07:00:00
2018-05-19 08:00:00
2018-05-19 09:00:00
2018-05-19 10:00:00
2018-05-19 11:00:00
2018-05-19 12:00:00
2018-05-19 13:00:00
2018-05-19 14:00:00
2018-05-19 15:00:00
2018-05-19 16:00:00
2018-05-19 17:00:00
2018-05-19 18:00:00
2018-05-19 19:00:00
2018-05-19 20:00:00
2018-05-19 21:00:00
2018-05-19 22:00:00
2018-05-19 23:00:00
2018-05-20 00:00:00
2018-05-20 01:00:00
2018-05-20 02:00:00
2018-05-20 03:00:00


2018-07-29 08:00:00
2018-07-29 09:00:00
2018-07-29 10:00:00
2018-07-29 11:00:00
2018-07-29 12:00:00
2018-07-29 13:00:00
2018-07-29 14:00:00
2018-07-29 15:00:00
2018-07-29 16:00:00
2018-07-29 17:00:00
2018-07-29 18:00:00
2018-07-29 19:00:00
2018-07-29 20:00:00
2018-07-29 21:00:00
2018-07-29 22:00:00
2018-07-29 23:00:00
2018-07-30 00:00:00
2018-07-30 01:00:00
2018-07-30 02:00:00
2018-07-30 03:00:00
2018-07-30 04:00:00
2018-07-30 05:00:00
2018-07-30 06:00:00
2018-07-30 07:00:00
2018-07-30 08:00:00
2018-07-30 09:00:00
2018-07-30 10:00:00
2018-07-30 11:00:00
2018-07-30 12:00:00
2018-07-30 13:00:00
2018-07-30 14:00:00
2018-07-30 15:00:00
2018-07-30 16:00:00
2018-07-30 17:00:00
2018-07-30 18:00:00
2018-07-30 19:00:00
2018-07-30 20:00:00
2018-07-30 21:00:00
2018-07-30 22:00:00
2018-07-30 23:00:00
2018-07-31 00:00:00
2018-07-31 01:00:00
2018-07-31 02:00:00
2018-07-31 03:00:00
2018-07-31 04:00:00
2018-07-31 05:00:00
2018-07-31 06:00:00
2018-07-31 07:00:00
2018-07-31 08:00:00
2018-07-31 09:00:00


2018-10-07 00:00:00
2018-10-07 01:00:00
2018-10-07 02:00:00
2018-10-07 03:00:00
2018-10-07 04:00:00
2018-10-07 05:00:00
2018-10-07 06:00:00
2018-10-07 07:00:00
2018-10-07 08:00:00
2018-10-07 09:00:00
2018-10-07 10:00:00
2018-10-07 11:00:00
2018-10-07 12:00:00
2018-10-07 13:00:00
2018-10-07 14:00:00
2018-10-07 15:00:00
2018-10-07 16:00:00
2018-10-07 17:00:00
2018-10-07 18:00:00
2018-10-07 19:00:00
2018-10-07 20:00:00
2018-10-07 21:00:00
2018-10-07 22:00:00
2018-10-07 23:00:00
2018-10-08 00:00:00
2018-10-08 01:00:00
2018-10-08 02:00:00
2018-10-08 03:00:00
2018-10-08 04:00:00
2018-10-08 05:00:00
2018-10-08 06:00:00
2018-10-08 07:00:00
2018-10-08 08:00:00
2018-10-08 09:00:00
2018-10-08 10:00:00
2018-10-08 11:00:00
2018-10-08 12:00:00
2018-10-08 13:00:00
2018-10-08 14:00:00
2018-10-08 15:00:00
2018-10-08 16:00:00
2018-10-08 17:00:00
2018-10-08 18:00:00
2018-10-08 19:00:00
2018-10-08 20:00:00
2018-10-08 21:00:00
2018-10-08 22:00:00
2018-10-08 23:00:00
2018-10-09 00:00:00
2018-10-09 01:00:00


2018-12-17 10:00:00
2018-12-17 11:00:00
2018-12-17 12:00:00
2018-12-17 13:00:00
2018-12-17 14:00:00
2018-12-17 15:00:00
2018-12-17 16:00:00
2018-12-17 17:00:00
2018-12-17 18:00:00
2018-12-17 19:00:00
2018-12-17 20:00:00
2018-12-17 21:00:00
2018-12-17 22:00:00
2018-12-17 23:00:00
2018-12-18 00:00:00
2018-12-18 01:00:00
2018-12-18 02:00:00
2018-12-18 03:00:00
2018-12-18 04:00:00
2018-12-18 05:00:00
2018-12-18 06:00:00
2018-12-18 07:00:00
2018-12-18 08:00:00
2018-12-18 09:00:00
2018-12-18 10:00:00
2018-12-18 11:00:00
2018-12-18 12:00:00
2018-12-18 13:00:00
2018-12-18 14:00:00
2018-12-18 15:00:00
2018-12-18 16:00:00
2018-12-18 17:00:00
2018-12-18 18:00:00
2018-12-18 19:00:00
2018-12-18 20:00:00
2018-12-18 21:00:00
2018-12-18 22:00:00
2018-12-18 23:00:00
2018-12-19 00:00:00
2018-12-19 01:00:00
2018-12-19 02:00:00
2018-12-19 03:00:00
2018-12-19 04:00:00
2018-12-19 05:00:00
2018-12-19 06:00:00
2018-12-19 07:00:00
2018-12-19 08:00:00
2018-12-19 09:00:00
2018-12-19 10:00:00
2018-12-19 11:00:00


### Merging weather and route 16 dataframes on 'Hour' and 'DayOfService'

In [135]:
WeatherCombined_df = pd.merge(df_16_1,
                 weather,
                 on=['Hour', 'DAYOFSERVICE'])

In [136]:
WeatherCombined_df = WeatherCombined_df[(WeatherCombined_df['JourneyTime'] > 0)]

In [137]:
weather.dtypes

weather_date            object
rain                   float64
temp                   float64
wetb                   float64
dewpt                  float64
vappr                  float64
rhum                     int64
msl                    float64
wdsp                     int64
wddir                    int64
ww                       int64
w                        int64
sun                    float64
vis                      int64
clht                     int64
clamt                    int64
Hour                     int64
DAYOFSERVICE    datetime64[ns]
dtype: object

In [138]:
WeatherCombined_df.drop(columns = ['dewpt','vappr','rhum','msl','wddir','ww','w','sun','clht'])

,index,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,...,RushHour,Weekend,Holiday,weather_date,rain,temp,wetb,wdsp,vis,clamt
0,114812203,2018-12-27,8591828,2,3669,15:49:10,15:49:10,15:49:32,15:50:02,1000115,...,0,0,0,2018-12-27 15:00:00,0.0,9.3,8.6,5,40000,7
1,114812204,2018-12-27,8591828,3,7349,15:50:03,15:50:03,15:51:25,15:51:25,1000115,...,0,0,0,2018-12-27 15:00:00,0.0,9.3,8.6,5,40000,7
2,114886799,2018-12-27,8591828,4,1631,15:51:08,15:51:08,15:52:46,15:52:46,1000115,...,0,0,0,2018-12-27 15:00:00,0.0,9.3,8.6,5,40000,7
3,114891220,2018-12-27,8591828,5,1632,15:51:26,15:51:26,15:52:53,15:52:53,1000115,...,0,0,0,2018-12-27 15:00:00,0.0,9.3,8.6,5,40000,7
4,114895616,2018-12-27,8591828,6,5053,15:51:49,15:51:49,15:53:05,15:53:05,1000115,...,0,0,0,2018-12-27 15:00:00,0.0,9.3,8.6,5,40000,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538743,81883,2018-01-01,5955277,44,1288,09:05:28,09:05:28,08:56:51,08:56:51,1001127,...,0,0,1,2018-01-01 08:00:00,0.0,3.9,3.2,16,25000,3
1538744,126570,2018-01-01,5955277,45,1289,09:05:57,09:05:57,08:57:08,08:57:08,1001127,...,0,0,1,2018-01-01 08:00:00,0.0,3.9,3.2,16,25000,3
1538745,126608,2018-01-01,5955277,46,1290,09:06:37,09:06:37,08:58:07,08:58:07,1001127,...,0,0,1,2018-01-01 08:00:00,0.0,3.9,3.2,16,25000,3
1538746,126879,2018-01-01,5955277,47,1291,09:07:40,09:07:40,08:59:21,08:59:21,1001127,...,0,0,1,2018-01-01 08:00:00,0.0,3.9,3.2,16,25000,3


### This csv will be read into the PredictionEvaluationModel notebook to carry out training and testing

In [1]:
WeatherCombined_df.to_csv('Weather_16_1_Combined.csv')

NameError: name 'WeatherCombined_df' is not defined